In [1]:
import tensorflow as tf, keras, numpy as np, cv2, os, VIT

In [2]:
model = keras.models.load_model(os.path.join('data', 'checkpoint', 'Final model', 'Final model.keras'))

c:\Users\Thoalfeqar\anaconda3\envs\VIT\Lib\site-packages\keras\src\layers\layer.py:391: UserWarning: `build()` was called on layer 'patch_encoder_1', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


In [3]:
optimized_model = tf.lite.Interpreter(os.path.join('data', 'checkpoint', 'Final model', 'Final model.tflite'))
optimized_model.allocate_tensors()
input_index = optimized_model.get_input_details()[0]['index']
output_index = optimized_model.get_output_details()[0]['index']
input_format = optimized_model.get_input_details()[0]['dtype']


def get_prediction_from_optimized_model(image, interpreter, input_index, output_index, input_format):
    image = np.array([image]).astype(input_format)
    interpreter.set_tensor(input_index, image)
    interpreter.invoke()
    return interpreter.get_tensor(output_index)
    
    

In [14]:
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(os.path.join('data', 'Haar cascades', 'haarcascade_frontalface_default.xml'))

labels = [
    'Thoalfeqar',
    'Not Thoalfeqar'
]


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    faces = face_cascade.detectMultiScale(frame, scaleFactor=1.3, minNeighbors=8, minSize = (112, 112))
    if len(faces) > 0 :
        for face in faces:
            x, y, w, h = face
            face_segment = cv2.resize(frame[y : y + h, x : x + w][:, :, ::-1], (112, 112))
            prediction = get_prediction_from_optimized_model(face_segment, optimized_model, input_index, output_index, input_format)[0]
            predicted_index = np.argmax(prediction)
            if(predicted_index == 0):
                if prediction[predicted_index] >= 0.97:
                    predicted_label = labels[predicted_index]
                else:
                    predicted_label = labels[1]
            else:
                predicted_label = labels[1]
                
            cv2.putText(frame, text = predicted_label, org = (x, y - 10), fontFace = cv2.FONT_HERSHEY_SIMPLEX, fontScale = 1, color = (0, 0, 255), thickness = 4)
            cv2.rectangle(frame, rec = face, color = (0, 0, 255), thickness = 1)


    cv2.imshow('hi', frame)
    if cv2.waitKey(1000//120) == ord('q'):
        break

cv2.destroyAllWindows()